In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
API_KEY = "0e8a6a2fca774fc5a0d0935de65ee23c"  # replace with your key
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [ ]:
# Function to get relative importance (weights)
def get_relative_importance():
    """
    BLS publishes relative importance data annually, but the series IDs for weights 
    are different and not easily accessible via the public API.
    Using the most recent known relative importance weights from BLS.
    """
    # Most recent relative importance weights (December 2024)
    # These are updated annually by BLS and represent the weight of each category in the CPI
    cpi_series = {
        'CUUR0000SA0': ('All items', 100.0),
        'CUUR0000SAF': ('Food', 13.37),
        'CUUR0000SAF11': ('Food at home', 8.80),
        'CUUR0000SEFV': ('Food away from home', 4.57),
        'CUUR0000SAH': ('Housing', 34.37),
        'CUUR0000SAH1': ('Shelter', 29.35),
        'CUUR0000SAH2': ('Fuels and utilities', 3.67),
        'CUUR0000SAH3': ('Household furnishings and operations', 1.35),
        'CUUR0000SAT': ('Transportation', 16.74),
        'CUUR0000SETA': ('New and used motor vehicles', 7.32),
        'CUUR0000SETB': ('Motor fuel', 3.38),
        'CUUR0000SAM': ('Medical care', 8.77),
        'CUUR0000SAA': ('Apparel', 2.51),
        'CUUR0000SAR': ('Recreation', 5.78),
        'CUUR0000SAE': ('Education and communication', 6.49),
        'CUUR0000SAG': ('Other goods and services', 3.01)
    }
    
    print("Using most recent relative importance weights from BLS (December 2024):")
    print("Note: BLS publishes updated relative importance weights annually.")
    print("For the most current weights, visit: https://www.bls.gov/cpi/tables/relative-importance/")
    
    return cpi_series

# Get the CPI series with weights
print("Setting up CPI categories and weights...")
cpi_series = get_relative_importance()
print("\nCPI Categories and Weights:")
for series_id, (desc, weight) in cpi_series.items():
    print(f"{desc}: {weight:.2f}%")

In [ ]:
# Function to calculate YoY inflation
def calculate_yoy_inflation(df):
    df['month'] = df['period'].str[1:].astype(int)
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str).str.zfill(2) + '-01')
    df = df.sort_values('date')
    df['yoy_inflation'] = (df['value'].pct_change(periods=12) * 100).round(2)
    return df

all_results = {}

for series_id, (description, weight) in cpi_series.items():
    payload = {
        "seriesid": [series_id],
        "startyear": "2023",
        "endyear": "2025",
        "registrationkey": API_KEY
    }
    
    response = requests.post(url, json=payload)
    data = response.json()
    if 'Results' in data and 'series' in data['Results'] and len(data['Results']['series']) > 0:
        series_data = data['Results']['series'][0]['data']
        
        df = pd.DataFrame(series_data)
        df['value'] = df['value'].astype(float)
        
        df = calculate_yoy_inflation(df)
        df.drop(columns=['year', 'period', 'periodName', 'latest', 'footnotes', 'month'], inplace=True)
        
        df['category'] = description
        df['weight'] = weight
        
        all_results[description] = df
    else:
        print(f"No data found for {description} ({series_id})")

In [ ]:
all_results.keys()

In [ ]:
all_results['All items']

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Combine all results into a single DataFrame for plotting
combined_df = pd.DataFrame()

for category, df in all_results.items():
    df_copy = df.copy()
    df_copy['category'] = category
    combined_df = pd.concat([combined_df, df_copy], ignore_index=True)

# Filter for 2024 and 2025 data only
combined_df = combined_df[combined_df['date'] >= '2024-01-01']

# Create interactive plot with Plotly
fig = go.Figure()

# Color palette for different categories
colors = px.colors.qualitative.Set3 + px.colors.qualitative.Pastel + px.colors.qualitative.Set1

# Add a trace for each category
for i, category in enumerate(all_results.keys()):
    category_data = combined_df[combined_df['category'] == category]
    
    # Determine if this should be visible by default (show main categories)
    visible = True if category in ['All items', 'Food', 'Housing', 'Transportation', 'Medical care'] else 'legendonly'
    
    fig.add_trace(go.Scatter(
        x=category_data['date'],
        y=category_data['yoy_inflation'],
        mode='lines+markers',
        name=f"{category} ({category_data['weight'].iloc[0]:.1f}%)",
        line=dict(width=3 if category == 'All items' else 2),
        marker=dict(size=6 if category == 'All items' else 4),
        visible=visible,
        hovertemplate='<b>%{fullData.name}</b><br>' +
                      'Date: %{x|%B %Y}<br>' +
                      'YoY Inflation: %{y:.1f}%<br>' +
                      '<extra></extra>'
    ))

# Update layout
fig.update_layout(
    title={
        'text': 'Year-over-Year Inflation by CPI Category (2024-2025)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis_title='Date',
    yaxis_title='Year-over-Year Inflation (%)',
    hovermode='x unified',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(size=10)
    ),
    width=1200,
    height=700,
    margin=dict(r=200),  # Make room for legend
    showlegend=True,
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgray'
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgray',
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black'
    )
)

# Add annotation explaining interaction
fig.add_annotation(
    text="Click legend items to show/hide categories<br>Double-click to isolate a single category",
    xref="paper", yref="paper",
    x=0.02, y=0.98,
    showarrow=False,
    font=dict(size=10, color="gray"),
    align="left"
)

fig.show()

# Display summary statistics
print("\nSummary of Latest Month's Inflation Rates:")
print("=" * 50)
latest_data = combined_df.loc[combined_df.groupby('category')['date'].idxmax()]
latest_data_sorted = latest_data.sort_values('yoy_inflation', ascending=False)

for _, row in latest_data_sorted.iterrows():
    print(f"{row['category']:<35} {row['yoy_inflation']:>6.1f}% (Weight: {row['weight']:>5.1f}%)")

In [ ]:
# Alternative: Static plot with matplotlib (uncomment to use)
"""
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(15, 10))

# Plot each category
for i, (category, df) in enumerate(all_results.items()):
    category_data = df[df['date'] >= '2024-01-01']
    
    # Highlight main categories with thicker lines
    linewidth = 3 if category == 'All items' else 2
    alpha = 1.0 if category in ['All items', 'Food', 'Housing', 'Transportation'] else 0.7
    
    plt.plot(category_data['date'], category_data['yoy_inflation'], 
             label=f"{category} ({df['weight'].iloc[0]:.1f}%)", 
             linewidth=linewidth, alpha=alpha, marker='o', markersize=4)

plt.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
plt.title('Year-over-Year Inflation by CPI Category (2024-2025)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Year-over-Year Inflation (%)', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Format x-axis to show months
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2))

plt.tight_layout()
plt.show()
"""